# Phase 1: LLAMA + MBPP Dataset Results Inspection

This notebook inspects Phase 1 LLAMA results to verify:
- Dataset files created correctly
- Model used (LLAMA-3.1-8B)
- Prompts formatted correctly
- Code generation working
- Evaluation results (pass/fail)
- Activations captured (31 layers)

In [ ]:
import pandas as pd
import os
from pathlib import Path
import glob
import json
import numpy as np

# Set pandas display options to show FULL content
pd.set_option('display.max_colwidth', None)  # Show full column content (NO LIMITS)
pd.set_option('display.max_columns', None)   # Show all columns
pd.set_option('display.width', None)         # Don't wrap to multiple lines
pd.set_option('display.max_rows', None)      # Show ALL rows (no limit)

print("✅ Pandas display options set to show FULL content")

In [ ]:
# Auto-discovery of Phase 1 LLAMA data
datasets_dir = "../data/phase1_0_llama/"
pattern = os.path.join(datasets_dir, "dataset_*.parquet")
matching_files = glob.glob(pattern)

if matching_files:
    # Sort by timestamp (filename)
    matching_files.sort(reverse=True)  # Most recent first
    print("=" * 80)
    print("PHASE 1: LLAMA + MBPP DATASET GENERATION")
    print("=" * 80)
    print(f"\n🔍 Found {len(matching_files)} dataset file(s)")
    for file in matching_files:
        file_size = os.path.getsize(file) / 1024  # KB
        print(f"  📁 {Path(file).name} ({file_size:.2f} KB)")
    
    # Use most recent file
    latest_file = matching_files[0]
    print(f"\n📄 Using most recent: {Path(latest_file).name}")
else:
    raise FileNotFoundError(f"❌ No dataset files found in {datasets_dir}")

In [ ]:
# Check activations directory
activations_dir = Path(datasets_dir) / "activations"

print("\n" + "=" * 80)
print("ACTIVATIONS DIRECTORY")
print("=" * 80)

if activations_dir.exists():
    print(f"\n✅ Activations directory exists: {activations_dir}")
    
    # Check subdirectories
    subdirs = [d for d in activations_dir.iterdir() if d.is_dir()]
    print(f"\n📁 Subdirectories ({len(subdirs)}):")
    for subdir in subdirs:
        files = list(subdir.glob("*.npz"))
        print(f"  - {subdir.name}/: {len(files)} activation files")
        if files:
            print(f"    Sample: {files[0].name}")
            
            # Check activation dimensions for LLAMA (should be 4096)
            sample_data = np.load(files[0])
            for key in sample_data.files:
                print(f"    Shape ({key}): {sample_data[key].shape}")
else:
    print(f"\n⚠️  Activations directory not found: {activations_dir}")

In [ ]:
# Load and display dataset
print(f"\n{'=' * 80}")
print(f"DATASET CONTENTS")
print("=" * 80)

df = pd.read_parquet(latest_file)

# Basic stats
print(f"\n📊 Statistics:")
print(f"  - Total records: {len(df)}")
print(f"  - Columns: {list(df.columns)}")

# Pass rate
if 'test_passed' in df.columns:
    n_passed = df['test_passed'].sum()
    pass_rate = n_passed / len(df) * 100
    print(f"\n✅ Pass Rate:")
    print(f"  - Passed: {n_passed}/{len(df)} ({pass_rate:.2f}%)")
    print(f"  - Failed: {len(df) - n_passed}/{len(df)} ({100 - pass_rate:.2f}%)")

# Generation time stats
if 'generation_time' in df.columns:
    print(f"\n⏱️  Generation Time:")
    print(f"  - Mean: {df['generation_time'].mean():.2f}s")
    print(f"  - Median: {df['generation_time'].median():.2f}s")
    print(f"  - Min: {df['generation_time'].min():.2f}s")
    print(f"  - Max: {df['generation_time'].max():.2f}s")

In [ ]:
# Display all records
print(f"\n{'=' * 80}")
print(f"ALL RECORDS:")
print("=" * 80)
display(df)

In [ ]:
# Show sample correct and incorrect generations
print("\n" + "=" * 80)
print("SAMPLE CORRECT GENERATION")
print("=" * 80)

if 'test_passed' in df.columns and df['test_passed'].any():
    correct_sample = df[df['test_passed'] == True].iloc[0]
    print(f"\n📋 Task ID: {correct_sample.get('task_id', 'N/A')}")
    print(f"\n📝 Prompt:")
    print(correct_sample.get('prompt', 'N/A')[:500] + "..." if len(str(correct_sample.get('prompt', ''))) > 500 else correct_sample.get('prompt', 'N/A'))
    print(f"\n💻 Generated Code:")
    print(correct_sample.get('generated_code', 'N/A'))
else:
    print("\n⚠️  No correct generations found")

In [ ]:
# Show sample incorrect generation
print("\n" + "=" * 80)
print("SAMPLE INCORRECT GENERATION")
print("=" * 80)

if 'test_passed' in df.columns and (~df['test_passed']).any():
    incorrect_sample = df[df['test_passed'] == False].iloc[0]
    print(f"\n📋 Task ID: {incorrect_sample.get('task_id', 'N/A')}")
    print(f"\n📝 Prompt:")
    print(incorrect_sample.get('prompt', 'N/A')[:500] + "..." if len(str(incorrect_sample.get('prompt', ''))) > 500 else incorrect_sample.get('prompt', 'N/A'))
    print(f"\n💻 Generated Code:")
    print(incorrect_sample.get('generated_code', 'N/A'))
    if 'error_message' in incorrect_sample and incorrect_sample['error_message']:
        print(f"\n❌ Error:")
        print(incorrect_sample['error_message'])
else:
    print("\n⚠️  No incorrect generations found")

In [ ]:
# Summary
print("\n" + "=" * 80)
print("OVERALL SUMMARY")
print("=" * 80)

print(f"\n📊 Dataset file: {Path(latest_file).name}")
print(f"📊 Total records: {len(df)}")
if 'test_passed' in df.columns:
    print(f"✅ Pass rate: {df['test_passed'].sum()}/{len(df)} ({df['test_passed'].mean()*100:.2f}%)")
print(f"📁 Activations exist: {'✅' if activations_dir.exists() else '❌'}")

# Check if activations have correct dimension for LLAMA (4096)
if activations_dir.exists():
    task_acts_dir = activations_dir / "task_activations"
    if task_acts_dir.exists():
        sample_files = list(task_acts_dir.glob("*.npz"))[:1]
        if sample_files:
            data = np.load(sample_files[0])
            for key in data.files:
                dim = data[key].shape[-1]
                expected = 4096  # LLAMA hidden size
                status = '✅' if dim == expected else '❌'
                print(f"{status} Activation dimension: {dim} (expected {expected} for LLAMA)")

print("\n🎉 Phase 1 LLAMA results inspection complete!")